## Build a Solar Atmosphere Using McWhirter and VALIIc Data

### Steps
1. Set parameters for model e.g. physical dimensions and grid size
2. Load VALIIIc and McWhirter Data
3. Interpolate Data
4. Regenerat model


In [1]:
import os
import numpy as np

import astropy.table
from astropy.table import Table
import astropy.units as u
from astropy.constants import k_B, m_p
import scipy.constants as asc
from astropy.table import Table


from scipy.interpolate import UnivariateSpline

from sacio import *
#alldat,modelinfo=read_sac_ascii('../../../configs/hydro/3D_128_spic_asc.ini')

In [2]:
alldat,modelinfo=read_sac_ascii('../../solar-testdata/3D_128_spic_bvertbg100G_asc.ini')



read in binary output 
rewrite as ascii
re-readascii file

In [3]:
print(modelinfo)
print(np.size(alldat))
print(np.shape(alldat))

#Display x co-ordinates here vertical section through atmosphere
print(alldat[48:80,64,64,0]/1.0e3)

('3D_mhd33                                                                        \n', 0, 0.0, 3, 7, 13, [128, 128, 128], '1.66667 0 1 -274 0 0 0', 'x y h m1 m2 m3 e b1 b2 b3 eb rhob bg1 bg2 bg3   gamma eta   grav1 grav2 grav3')
33554432
(128, 128, 128, 16)
[ 2444.44  2488.89  2533.33  2577.78  2622.22  2666.67  2711.11  2755.56
  2800.    2844.44  2888.89  2933.33  2977.78  3022.22  3066.67  3111.11
  3155.56  3200.    3244.44  3288.89  3333.33  3377.78  3422.22  3466.67
  3511.11  3555.56  3600.    3644.44  3688.89  3733.33  3777.78  3822.22]


### Extracting some of the data fields

In [4]:
dens=alldat[:,:,:,3]

#X=alldat[0,64,:,:]/1.0e6
#Y=alldat[1,64,:,:]/1.0e6
#vx=alldat[4,64,:,:]/(alldat[3,64,:,:]+alldat[12,64,:,:])
#vy=100000*alldat[5,64,:,:]/(alldat[3,64,:,:]+alldat[12,64,:,:])
#vz=100000*alldat[6,64,:,:]/(alldat[3,64,:,:]+alldat[12,64,:,:])
#rho=(alldat[3,64,:,:])*1.0e12
#e=alldat[7,64,:,:]
#vx=alldat[4,64,:,:]
#dens=(alldat[3,64,:,:]+alldat[12,64,:,:])
#bx=alldat[8,64,:,:]*sqrt(mu)*1.0e4
#by=alldat[9,64,:,:]*sqrt(mu)*1.0e4
#bz=alldat[10,64,:,:]*sqrt(mu)*1.0e4
print(alldat[48:80,64,64,12])

[  2.91198000e-11   2.36558000e-11   1.96920000e-11   1.68091000e-11
   1.47050000e-11   1.31623000e-11   1.20245000e-11   1.11788000e-11
   1.05441000e-11   1.00620000e-11   9.69031000e-12   9.39868000e-12
   9.16534000e-12   8.97443000e-12   8.81469000e-12   8.67782000e-12
   8.55799000e-12   8.45085000e-12   8.35333000e-12   8.26321000e-12
   8.17879000e-12   8.09895000e-12   8.02276000e-12   7.94964000e-12
   7.87904000e-12   7.81070000e-12   7.74426000e-12   7.67958000e-12
   7.61650000e-12   7.55485000e-12   7.49460000e-12   7.43560000e-12]


In [5]:
from valmcw import *
data=val3c()
#data.sort('Z')
print(data)

   Z            rho            p         T    
   km         g / cm3      dyn / cm2     K    
-------- ----------------- --------- ---------
   -75.0         3.192e-07  179000.0    8320.0
   -50.0          3.08e-07  157500.0    7610.0
   -25.0         2.949e-07  136800.0    6910.0
     0.0         2.727e-07  117200.0    6420.0
    50.0         2.152e-07   82740.0    5840.0
   100.0         1.606e-07   58040.0    5455.0
   150.0          1.15e-07   39260.0    5180.0
   250.0         5.413e-08   16910.0    4780.0
   350.0         2.334e-08    6798.0    4465.0
   450.0         9.327e-09    2569.0    4220.0
     ...               ...       ...       ...
  3400.0 1.93825170663e-15     0.118  444000.0
  3900.0  1.6575187938e-15     0.115  506000.0
  5860.0 1.20612032167e-15     0.107  647000.0
  8790.0 9.13252940484e-16    0.0988  789000.0
 13200.0 6.93504448156e-16    0.0891  937000.0
 19800.0 5.18471878702e-16    0.0782 1100000.0
 29700.0 3.81307630549e-16    0.0664 1270000.0
 44500.0 2.71

In [9]:
__all__ = ['read_VAL3c_MTW', 'interpolate_atmosphere', 'get_spruit_hs', 'vertical_profile']

In [10]:
#if 1D or 2D set unused dimensions to 0, and unrequired xyz limits to 1. 
Nxyz = [128,128,128] # 3D grid
xyz_SI = [-1e6,1e6,-1e6,1e6,3.5e4,1.6e6] # xyz limits SI/CGS units 
xyz  = [-1.27*u.Mm,1.27*u.Mm,-1.27*u.Mm,1.27*u.Mm,0.*u.Mm,8.62*u.Mm] #grid size
#xyz.unit=u.Mm
#xyz=np.array([-1e6,1e6,-1e6,1e6,3.5e4,1.6e6])
#xyz = [-1*u.Mm,1e6,-1*u.Mm,1*u.Mm,0.035*u.Mm,1.6*u.Mm] # xyz limits SI/CGS units  
#xyz=xyz_SI

In [11]:
"""
get_coords returns a non-dimensional dictionary describing the domain
coordinates.
"""
dz=(xyz[5]-xyz[4])/(Nxyz[2]-1)
#Z    = u.Quantity(np.linspace(xyz[4].value, xyz[5].value, Nxyz[2]), unit=xyz.unit)
Z    = u.Quantity(np.linspace(xyz[4].value, xyz[5].value, Nxyz[2]))
#Zext = u.Quantity(np.linspace(Z.min().value-4.*dz.value, Z.max().value+4.*dz.value, Nxyz[2]+8), unit=Z.unit)
Zext = u.Quantity(np.linspace(Z.min().value-4.*dz.value, Z.max().value+4.*dz.value, Nxyz[2]+8))
coords = {
              'dx':(xyz[1]-xyz[0])/(Nxyz[0]-1),
              'dy':(xyz[3]-xyz[2])/(Nxyz[1]-1),
              'dz':(xyz[5]-xyz[4])/(Nxyz[2]-1),
              'xmin':xyz[0],
              'xmax':xyz[1],
              'ymin':xyz[2],
              'ymax':xyz[3],
              'zmin':xyz[4],
              'zmax':xyz[5],
              'Z':Z,
              'Zext':Zext
            }

In [12]:
#identify location of source data files
__files__=''
homedir = os.environ['HOME']
cwd = os.path.dirname(__files__)
#cwd = homedir+'/Dropbox/multi_tube/python/allpapers/'
VAL_file = os.path.join(cwd, 'VALIIIC.dat')
MTW_file = os.path.join(cwd, 'mcwhirter.dat')

filenames = [VAL_file, MTW_file]
# uncomment and switch to l_const/l_sqrt/l_linear/l_square as required  
#logical_pars['l_square'] = True 

In [13]:
#============================================================================
    # Dimensional units in terms of SI
#============================================================================
scales   = {
            'length':         1e2*u.Mm,
            'density':        1e-4*u.kg/u.m**3,
            'velocity':       1e2*u.m/u.s,
            'temperature':    1.0*u.K, 
            'magnetic':       1e-3*u.T #mT
           }


scales['energy density'] = scales['density'] * scales['velocity']**2
scales['time'] = scales['length'] / scales['velocity'] 
scales['mass'] = scales['density'] * scales['length']**3 
scales['force density'] = scales['density'] * scales['velocity'] / \
scales['time'] #D momentum/Dt force density balance 

In [14]:
print(5*scales['temperature'])

print(coords)

5.0 K
{'zmax': <Quantity 8.62 Mm>, 'zmin': <Quantity 0.0 Mm>, 'dz': <Quantity 0.0678740157480315 Mm>, 'dx': <Quantity 0.02 Mm>, 'dy': <Quantity 0.02 Mm>, 'xmin': <Quantity -1.27 Mm>, 'Zext': <Quantity [-0.27149606,-0.20362205,-0.13574803,-0.06787402, 0.        ,
            0.06787402, 0.13574803, 0.20362205, 0.27149606, 0.33937008,
            0.40724409, 0.47511811, 0.54299213, 0.61086614, 0.67874016,
            0.74661417, 0.81448819, 0.8823622 , 0.95023622, 1.01811024,
            1.08598425, 1.15385827, 1.22173228, 1.2896063 , 1.35748031,
            1.42535433, 1.49322835, 1.56110236, 1.62897638, 1.69685039,
            1.76472441, 1.83259843, 1.90047244, 1.96834646, 2.03622047,
            2.10409449, 2.1719685 , 2.23984252, 2.30771654, 2.37559055,
            2.44346457, 2.51133858, 2.5792126 , 2.64708661, 2.71496063,
            2.78283465, 2.85070866, 2.91858268, 2.98645669, 3.05433071,
            3.12220472, 3.19007874, 3.25795276, 3.32582677, 3.39370079,
            3.461

In [15]:
print (xyz[1]-xyz[0])/(Nxyz[0]-1)

0.02 Mm


In [16]:
from scipy.interpolate import UnivariateSpline
s=0.25
hdata = np.array(u.Quantity(data['Z']).to(u.m))
# interpolate total pressure, temperature and density profiles
pdata_f = UnivariateSpline(hdata,np.array(np.log(data['p'])),k=1, s=s)
Tdata_f = UnivariateSpline(hdata,np.array(np.log(data['T'])),k=1, s=s)
rdata_f = UnivariateSpline(hdata,np.array(np.log(data['rho'])),k=1, s=s)
#s=0.0 to ensure all points are strictly used for ionisation state
#muofT_f = UnivariateSpline(hdata,np.array(np.log(data['mu'])),k=1, s=0.0)
#print pdata_f(Z)
#print Z
print scales['temperature']
print(Tdata_f(hdata))
#print((data['Z']))
print Nxyz[0]
outdata = Table([np.zeros((Nxyz[0])),np.zeros((Nxyz[0])),np.zeros((Nxyz[0])),np.zeros((Nxyz[0]))],names=['Z', 'T','p','rho'])
outdata['Z'] = Z
#outdata['p'] = np.exp(pdata_f(Z.to(u.m))) 
#outdata['p'] = np.exp(pdata_f(Z.to(u.cm))) * data['p'].unit
outdata['p'] = np.exp(pdata_f(Z*1.0e8)) * data['p'].unit
#outdata['T'] = np.exp(Tdata_f(Z.to(u.m))) * data['T'].unit
#outdata['T'] = np.exp(Tdata_f()) * data['T'].unit 
outdata['T'] = np.exp(Tdata_f(Z*1.0e6)) * data['T'].unit
#outdata['rho'] = np.exp(rdata_f(Z.to(u.m))) * data['rho'].unit
#outdata['rho'] = np.exp(rdata_f()) * data['rho'].unit 
outdata['rho'] = np.exp(rdata_f(Z*1.0e8)) * data['rho'].unit 
#outdata['mu'] = np.exp(muofT_f(Z.to(u.m))) * u.one



1.0 K
[  8.93174722   8.8930591    8.85437097   8.81568285   8.73830661
   8.66093036   8.58355412   8.42880163   8.27404914   8.3422776
   8.38662609   8.41391748   8.44803171   8.48214594   8.51626017
   8.5503744    8.61860286   8.63169179   8.65132519   8.67357637
   8.70368091   8.72985877   8.75603663   8.79137675   8.81493682
   8.86205697   8.89870598   8.91572159   8.92252783   8.93142831
   8.93666388   8.93928167   8.94189945   9.31143004   9.39061517
   9.53115427   9.81223248   9.82559102   9.85898735   9.91910075
  10.12615803  10.11589002  10.10818901  10.1017715   10.09971789
  10.26595525  10.4321926   10.55687061  10.80622664  11.3645516
  11.81121156  12.63396845  12.72829267  12.88524633  12.96372316
  13.04219999  13.23839206  13.33382878  13.47649692  13.69122953
  14.01259806  14.06745975  14.1494752   14.27305255  14.45703316]
128


In [17]:
print outdata['rho'][1]

#VAL3c = np.loadtxt(filenames[0])[::-1,:]
#VAL3c[:,0] *= 1e3 #km -> m
#VAL3c[:,1] *= 1e3 #g/cm^3 -> kg/m^3
#VAL3c[:,2] /= 1e1 #dyne/cm^2 -> Pascals(N/m^2)
#muTV=4.0/(3*0.74+1+VAL3c[:,6]/VAL3c[:,5])
#nzv=VAL3c[:,0].size
#MTW =np.loadtxt(filenames[1])[::-1,:]
#MTW[:,0] *= 1e3 #km -> m
#MTW[:,1] *= 1   #Kelvin
#MTW[:,2] /= 1e1 #dyne/cm^2 -> Pascals(N/m^2)
#nzm=MTW[:,0].size
#Combine both sets into single array
#hdata=np.zeros(nzv+nzm)
#hdata[0:nzv]=VAL3c[:,0]
#hdata[nzv:nzv+nzm]=MTW[:,0]
#hdata /= scales['length']
    
#pdata=np.zeros(nzv+nzm)
#pdata[0:nzv]=VAL3c[:,2]
#pdata[nzv:nzv+nzm]=MTW[:,2]
#pdata /= scales['energy density']
    
#Tdata=np.zeros(nzv+nzm)
#Tdata[0:nzv]=VAL3c[:,3]
#Tdata[nzv:nzv+nzm]=MTW[:,1] 
#Tdata /= scales['temperature']
    
#rdata=np.zeros(nzv+nzm)
#rdata[0:nzv]=VAL3c[:,1]
#rdata /= scales['density']   
#MTW[:,2], kB, mp and mu are in code units so no rescale needed
#rdata[nzv:nzv+nzm] = MTW[:,2]/physicalconstants['boltzmann']/MTW[:,1] *physicalconstants['proton_mass']*physicalconstants['mu']
    
#muofT = np.zeros(nzv+nzm) # mean molecular weight
#muofT[0:nzv] = muTV
#muofT[nzv:nzv+nzm] = physicalconstants['mu']
# interpolate total pressure, temperature and density profiles
#pdata_f = UnivariateSpline(hdata,np.log(pdata),k=1, s=0.25)
#Tdata_f = UnivariateSpline(hdata,np.log(Tdata),k=1, s=0.25)
#rdata_f = UnivariateSpline(hdata,np.log(rdata),k=1, s=0.25)
#s=0.0 to ensure all points are strictly used for ionisation state
#muofT_f = UnivariateSpline(hdata,np.log(muofT),k=1, s=0.0)
#pdata_i = np.exp(pdata_f(Z))
#Tdata_i = np.exp(Tdata_f(Z))
#rdata_i = np.exp(rdata_f(Z))
#muofT_i = np.exp(muofT_f(Z))

#source_data = [VAL3c,MTW] 

1.1891867963e-15


In [18]:
print Tdata_f(Z)

[ 8.81568285  8.81568275  8.81568264  8.81568254  8.81568243  8.81568233
  8.81568222  8.81568212  8.81568201  8.81568191  8.8156818   8.81568169
  8.81568159  8.81568148  8.81568138  8.81568127  8.81568117  8.81568106
  8.81568096  8.81568085  8.81568075  8.81568064  8.81568054  8.81568043
  8.81568033  8.81568022  8.81568012  8.81568001  8.81567991  8.8156798
  8.8156797   8.81567959  8.81567949  8.81567938  8.81567928  8.81567917
  8.81567907  8.81567896  8.81567886  8.81567875  8.81567865  8.81567854
  8.81567844  8.81567833  8.81567823  8.81567812  8.81567802  8.81567791
  8.81567781  8.8156777   8.8156776   8.81567749  8.81567739  8.81567728
  8.81567718  8.81567707  8.81567697  8.81567686  8.81567676  8.81567665
  8.81567655  8.81567644  8.81567634  8.81567623  8.81567613  8.81567602
  8.81567592  8.81567581  8.81567571  8.8156756   8.8156755   8.81567539
  8.81567529  8.81567518  8.81567508  8.81567497  8.81567487  8.81567476
  8.81567466  8.81567455  8.81567445  8.81567434  8.

## Recalculate density values

don't forget to adjust modelinfo values!

In [19]:
for i3 in range(Nxyz[2]):
    for i2 in range(Nxyz[1]):
        for i1 in range(Nxyz[0]):
            alldat[i1,i2,i3,12]=1000*outdata['rho'][i1]
            alldat[i1,i2,i3,0]=1000*outdata['Z'][i1]

In [22]:
print(alldat[48:80,64,64,0]/1.0e3)
print(alldat[48:80,64,64,12])

[ 3.25795276  3.32582677  3.39370079  3.4615748   3.52944882  3.59732283
  3.66519685  3.73307087  3.80094488  3.8688189   3.93669291  4.00456693
  4.07244094  4.14031496  4.20818898  4.27606299  4.34393701  4.41181102
  4.47968504  4.54755906  4.61543307  4.68330709  4.7511811   4.81905512
  4.88692913  4.95480315  5.02267717  5.09055118  5.1584252   5.22629921
  5.29417323  5.36204724]
[  2.29525398e-15   2.04190059e-15   1.81651270e-15   1.61600346e-15
   1.43762670e-15   1.27893943e-15   1.13776828e-15   1.01217981e-15
   9.00453963e-16   8.01060577e-16   7.12638375e-16   6.33976341e-16
   5.63997133e-16   5.01742330e-16   4.46359300e-16   3.97089527e-16
   3.53258222e-16   3.14265079e-16   2.79576054e-16   2.48716052e-16
   2.21262422e-16   1.96839162e-16   1.75111776e-16   1.55782690e-16
   1.38587176e-16   1.23289727e-16   1.09680832e-16   9.75741061e-17
   8.68037379e-17   7.72222182e-17   6.86983203e-17   6.11153023e-17]


## write data to outputfile

In [21]:
write_sac_ascii('test.ini',alldat,modelinfo)

## hydrostatic pressure balance